In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio

warnings.filterwarnings('ignore')
nest_asyncio.apply()
_ = load_dotenv(find_dotenv())

## Setup neo4j

In the terminal, type:
```
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

From here, you can open the db at http://localhost:7474/. On this page, you will be asked to sign in. Use the default username/password of neo4j and neo4j.

Once you login for the first time, you will be asked to change the password.

After this, you are ready to create your first property graph!

In [2]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="llamaindex",
    url="bolt://localhost:7687",
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


## Setup Qdrant Client
In the terminal, type:
```
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant
```
This will spin up a Qdrant vector database instance

In [3]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(host="localhost", port=6333)
aclient = AsyncQdrantClient(host="localhost", port=6333)

# delete collection if it exists
if client.collection_exists("PaulGraham"):
    client.delete_collection("PaulGraham")

vector_store = QdrantVectorStore(
    "Titus_Resume",
    client = client,
    aclient = aclient,
    fastembed_sparse_model="Qdrant/bm42-all-minilm-l6-v2-attentions",
)

Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

## Ingest documents

In [4]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader(input_files=["../../../data/paul_graham_essay.txt"]).load_data()

## Setup PropertyGraphIndex

In [5]:
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.indices import PropertyGraphIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm = Ollama("llama3.2")
Settings.embed_model = OllamaEmbedding("nomic-embed-text")

In [6]:
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    ImplicitPathExtractor
)

simple_path_extractor = SimpleLLMPathExtractor(
    max_paths_per_chunk=10,
    num_workers=4,
)
implicit_path_extractor = ImplicitPathExtractor()

In [7]:
index = PropertyGraphIndex.from_documents(
    docs,
    property_graph = graph_store,
    vector_store = vector_store,
    embed_kg_nodes = True,
    kg_extractors = [
        simple_path_extractor,
        implicit_path_extractor
    ]
)

In [ ]:
query_engine = index.as_query_engine()
query_engine.query("What is Titus' experience in data science?")